In [14]:
# coding: utf-8

# loads packages
import glob
import os
import random
import logging
import pandas
import numpy
import pickle
import gensim
import operator
import re
from random import shuffle


from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from typing import Iterable

from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import Doc2Vec
from nltk import tokenize, casual_tokenize
import nltk
from zipfile import ZipFile
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.corpus import stopwords

In [8]:
model = Doc2Vec.load("/home/jcai/geometry_of_law/doc2vec_v50k_d200_shuffled_opinion/ALL_opinion.d2v")

In [13]:
model['1966-2778-maj.txt']

array([-0.8388337 ,  0.01929922,  1.6417388 ,  1.6680253 , -1.1599753 ,
       -0.10476294,  1.2732133 ,  1.62956   ,  0.24972002,  0.24431233,
        1.6496967 , -0.77859443,  0.02725018, -1.405822  ,  0.6484543 ,
        0.19503969,  2.808611  , -1.5106868 ,  0.26638207,  0.3649082 ,
       -0.57813984,  0.5644977 ,  2.1251717 , -0.29678583,  0.87085897,
       -1.056622  , -0.4768481 , -1.41403   , -0.96640956, -0.35869974,
        1.0244193 , -1.55271   ,  1.9209219 ,  1.0607615 , -0.3988742 ,
       -0.5231457 ,  1.2117379 , -0.7968446 ,  0.9742706 ,  0.8193205 ,
        0.68929774, -2.516063  , -0.72803897,  0.01112037,  1.6645523 ,
       -1.6293631 ,  1.3326203 , -1.2277672 , -0.34811673,  2.66561   ,
        0.096725  ,  0.42410973, -0.43092084, -0.40046343, -0.633903  ,
       -0.01690531, -0.47565016, -1.4355826 , -0.76338327,  0.335635  ,
        0.38639477, -0.27080134, -1.2570205 ,  0.9702618 , -1.5572479 ,
        2.4277866 , -0.4489372 ,  0.8995244 , -2.604513  ,  1.68

In [12]:
model.docvecs.doctags

{'X3FE6A_contentMajOp_WILLIAMS.txt': Doctag(offset=0, word_count=1958, doc_count=1),
 'XABJ4O_contentDisOp1_GIBBONS.txt': Doctag(offset=1, word_count=1140, doc_count=1),
 'X43FR8_contentMajOp_PER CURIAM..txt': Doctag(offset=2, word_count=71, doc_count=1),
 'X42N9T_contentMajOp_PER CURIAM.txt': Doctag(offset=3, word_count=576, doc_count=1),
 'X31DE5_contentMajOp_FAGG.txt': Doctag(offset=4, word_count=336, doc_count=1),
 'X39TFI_contentMajOp_WALD.txt': Doctag(offset=5, word_count=3401, doc_count=1),
 'X3N3ML_contentMajOp_ADAMS.txt': Doctag(offset=6, word_count=1785, doc_count=1),
 'X12I4I0003_contentMajOp_HUG.txt': Doctag(offset=7, word_count=2189, doc_count=1),
 'X121BHQ003_contentMajOp_FRIEDMAN.txt': Doctag(offset=8, word_count=1522, doc_count=1),
 'X41DKV_contentDisOp1_HUTCHESON.txt': Doctag(offset=9, word_count=92, doc_count=1),
 'X45L5E_contentMajOp_TUTTLE.txt': Doctag(offset=10, word_count=1124, doc_count=1),
 'X18SPSS003_contentMajOp_MURPHY.txt': Doctag(offset=11, word_count=358, 

In [16]:
import pandas as pd

In [118]:
master_df = pd.read_csv("/home/jcai/geometry_of_law/Encyclopedia Entry/master_dataframe.csv")

In [18]:
master_df.columns

Index(['docname', 'judge_name', 'party', 'year', 'circuit', 'circuit-year',
       'big-issue', 'detailed-issue', 'circuit-big-issue', 'year-big-issue',
       'circuit-big-issue-year', 'judge-year', 'decade', 'judge-decade',
       'court-decade'],
      dtype='object')

In [22]:
master_df["caseid"] = [x.split("_")[0] for x in master_df["docname"]]

In [25]:
outcome_data = pd.read_csv('/home/jcai/geometry_of_law/data_and_dictionary/circuit_case_sc_decision_map_full.csv')

In [89]:
outcome_data.columns

Index(['cc_caseid', 'caseDisposition_binary', 'caseId', 'docketId',
       'caseIssuesId', 'voteId', 'dateDecision', 'usCite', 'sctCite',
       'ledCite', 'lexisCite', 'term', 'docket', 'caseName', 'lcDisposition',
       'lc_case_outcome_disposition', 'caseDisposition_org',
       'justice_outcome_disposition', 'justice', 'justiceName', 'file',
       'cc_judge', 'cc_year'],
      dtype='object')

In [125]:
import numpy as np

In [126]:
def return_file_match(i):
    try:
        return outcome_data["caseId"][i].split("-")[0]+"-"+outcome_data["file"][i].split(".")[0]
    except:
        return np.nan

In [127]:
outcome_data["file_match"] = [return_file_match(i) for i in outcome_data.index]

In [131]:
master_df["caseid"] = [master_df["docname"][i].split("_")[0] for i in master_df.index]

In [133]:
master_df = master_df.merge(outcome_data, right_on = "cc_caseid", left_on = "caseid", how = "left")

In [136]:
master_df.to_csv("/home/jcai/geometry_of_law/data_and_dictionary/link_to_sc_decision.csv")

In [96]:
zfile_sc = ZipFile('/data/Data/Supreme_Court_Cases/SC_Cases_1880_2016.zip', allowZip64 = True)

In [100]:
namelist = zfile_sc.namelist()

In [115]:
matching = [s for s in namelist if "1993-7941" in s]

In [116]:
matching

['SC_Cases_1880_2016/1993/1993-7941-concur.txt',
 'SC_Cases_1880_2016/1993/1993-7941-dis.txt',
 'SC_Cases_1880_2016/1993/1993-7941-maj.txt']